In [1]:
%cd '..'

/mnt/scratch/v_kulikov/histonedb_Vladimir/CURATED_SET


In [2]:
import inspect
import os
import re
import sys

import pandas as pd
from Bio import Entrez, SeqIO
from curated_set_services import CuratedSet

In [3]:
# Load data from histones.csv
curated_set = CuratedSet()
cs = curated_set

cs.data.shape, cs.data.columns

((564, 16),
 Index(['accession', 'type', 'variant_group', 'variant', 'doublet', 'gi',
        'ncbi_gene_id', 'hgnc_gene_name', 'taxonomy_id', 'organism', 'phylum',
        'class', 'taxonomy_group', 'info', 'references', 'sequence'],
       dtype='object'))

In [4]:
list(cs.has_duplicates())

[]

In [5]:
cs.data[cs.data["accession"] == "U84115.1"]

,accession,type,variant_group,variant,doublet,gi,ncbi_gene_id,hgnc_gene_name,taxonomy_id,organism,phylum,class,taxonomy_group,info,references,sequence


In [6]:
cs.data.head()

,accession,type,variant_group,variant,doublet,gi,ncbi_gene_id,hgnc_gene_name,taxonomy_id,organism,phylum,class,taxonomy_group,info,references,sequence
NP_505463.1,NP_505463.1,H2A,cH2A,cH2A_(Animals),,17562014,,,6239,Caenorhabditis elegans,Nematoda,Chromadorea,,,26989147 22650316(?),MSGRGKGGKAKTGGKAKSRSSRAGLQFPVGRLHRILRKGNYAQRVG...
EEC09557.1,EEC09557.1,H2A,cH2A,cH2A_(Animals),,215500063,,,6945,Ixodes scapularis,Arthropoda,Arachnida,,,26989147 22650316(?),MSGRGKGGKVKGKSKTRSSRAGLQFPVGRIHRLLRKGNYAERVGAG...
NP_724343.1,NP_724343.1,H2A,cH2A,cH2A_(Animals),,24585673,,,7227,Drosophila melanogaster,Arthropoda,Insecta,,,26989147 22650316(?),MSGRGKGGKVKGKAKSRSNRAGLQFPVGRIHRLLRKGNYAERVGAG...
XP_001119899.1,XP_001119899.1,H2A,cH2A,cH2A_(Animals),,110764935,,,7460,Apis mellifera,Arthropoda,Insecta,,,26989147 22650316(?),MSGRGKGGKAKAKAKSRSNRAGLQFPVGRIHRLLRKGNYAERVGAG...
EDO48405.1,EDO48405.1,H2A,cH2A,cH2A_(Animals),,156227602,,,45351,Nematostella vectensis,Cnidaria,Anthozoa,,,26989147 22650316(?),MSGRGKGKAKGTKSKTRSSRAGLQFPVGRIHRHLRKGNYAERVGAG...


In [7]:
cs.data.shape

(564, 16)

In [8]:
late_h2b_accessions = [
    "U84115.1",
]

In [9]:
df = pd.DataFrame(
    {
        "accession": late_h2b_accessions,
        "type": ["H2B"] * len(late_h2b_accessions),
        "variant_group": ["CS_H2B"] * len(late_h2b_accessions),
        "variant": ["CS_H2B"] * len(late_h2b_accessions),
        "references": ["9032246"] * len(late_h2b_accessions),
    }
)
df.index = df.accession
df.shape, df.columns

((1, 5),
 Index(['accession', 'type', 'variant_group', 'variant', 'references'], dtype='object'))

In [10]:
df.head()

,accession,type,variant_group,variant,references
accession,,,,,
U84115.1,U84115.1,H2B,CS_H2B,CS_H2B,9032246


In [11]:
cs.data = pd.concat([cs.data, df]).fillna("")
cs.data.shape, list(cs.has_duplicates())

((565, 16), [])

In [12]:
cs.data.tail()

,accession,type,variant_group,variant,doublet,gi,ncbi_gene_id,hgnc_gene_name,taxonomy_id,organism,phylum,class,taxonomy_group,info,references,sequence
NP_006017.1,NP_006017.1,H1,H1.10,H1.10_(Homo_sapiens)__???,,,8971.0,H1-10,9606,Homo sapiens,Chordata,Mammalia,Mammalia,,26689747,MSVELEEALPVTTAEGMAKKVTKAGGSAALSPSKKRKNSKKKNQPG...
NP_722575.1,NP_722575.1,H1,H1.8,H1.8_(Homo_sapiens)__???,,,132243.0,H1-8,9606,Homo sapiens,Chordata,Mammalia,Mammalia,,26689747,MAPGSVTSDISPSSTSTAGSSRSPESEKPGPSHGGVPPGGPSHSSL...
NP_001295191.1,NP_001295191.1,H1,H1.8,H1.8_(Homo_sapiens)__???,,,132243.0,H1-8,9606,Homo sapiens,Chordata,Mammalia,Mammalia,,26689747,MAPATAPRRAGEAKGKGPKKPSEAKEDPPNVGKVKKAAKRPAKVQK...
NP_005316.1,NP_005316.1,H1,H1.1,H1.1_(Homo_sapiens)__???,,,3024.0,H1-1,9606,Homo sapiens,Chordata,Mammalia,Mammalia,,26689747,MSETVPPAPAASAAPEKPLAGKKAKKPAKAAAASKKKPAGPSVSEL...
U84115.1,U84115.1,H2B,CS_H2B,CS_H2B,,,,,,,,,,,9032246,


In [13]:
cs.data[cs.data["variant"] == "CS_H2B"].shape

(1, 16)

In [14]:
cs.update_accession_version()
cs.data = cs.data.set_index(cs.data.accession.values)

/home/l_singh/.conda/envs/histdb_env/lib/python3.8/site-packages/Bio/Entrez/__init__.py:658: UserWarning: 
Email address is not specified.

To make use of NCBI's E-utilities, NCBI requires you to specify your
email address with each request.  As an example, if your email address
is A.N.Other@example.com, you can specify it as follows:
   from Bio import Entrez
   Entrez.email = 'A.N.Other@example.com'
In case of excessive usage of the E-utilities, NCBI will attempt to contact
a user at the email address provided before blocking access to the
E-utilities.
  warnings.warn(
/home/l_singh/.conda/envs/histdb_env/lib/python3.8/site-packages/Bio/GenBank/__init__.py:1143: BiopythonParserWarning: Dropping bond qualifier in feature location
  warnings.warn(


Mismatch: 520   519
Mismatch: 520   519
Mismatch: 520   519
Mismatch: 520   519
Mismatch: 520   519
Mismatch: 520   519
Mismatch: 520   519
Mismatch: 520   519
Mismatch: 520   519
Mismatch: 520   519


In [15]:
cs.data[cs.data["variant"] == "CS_H2B"].shape

(1, 16)

In [16]:
curated_set.update_taxids(blank_data=True)

Unexpected error: <class 'urllib.error.HTTPError'>, Retrying, attempt 0
Unexpected error: <class 'urllib.error.HTTPError'>, Retrying, attempt 1
Unexpected error: <class 'urllib.error.HTTPError'>, Retrying, attempt 2
Unexpected error: <class 'urllib.error.HTTPError'>, Retrying, attempt 3
Unexpected error: <class 'urllib.error.HTTPError'>, Retrying, attempt 4
Unexpected error: <class 'urllib.error.HTTPError'>, Retrying, attempt 5
Unexpected error: <class 'urllib.error.HTTPError'>, Retrying, attempt 6
Unexpected error: <class 'urllib.error.HTTPError'>, Retrying, attempt 7
Unexpected error: <class 'urllib.error.HTTPError'>, Retrying, attempt 8
Unexpected error: <class 'urllib.error.HTTPError'>, Retrying, attempt 9
FATAL ERROR could not get seqs from NCBI after 10 attempts for U84115.1. Will return empty list!


ValueError: Length of values (0) does not match length of index (1)

In [ ]:
cs.data[cs.data["variant"] == "CS_H2B"].shape

In [ ]:
cs.data[cs.data["variant"] == "CS_H2B"]

## Updating sequences for late_H2B.3

In [21]:
cs.data[cs.data['sequence'] == '']

,accession,type,variant_group,variant,doublet,gi,ncbi_gene_id,hgnc_gene_name,taxonomy_id,organism,phylum,class,taxonomy_group,info,references,sequence
P16888.2,P16888.2,H2B,late_H2B,late_H2B.1,,,,,7668,Strongylocentrotus purpuratus,Echinodermata,Echinoidea,,,3697096,


In [22]:
cs.update_sequence(blank_data=True)

Fetching 1 seqs


/home/l_singh/.conda/envs/histdb_env/lib/python3.8/site-packages/Bio/Entrez/__init__.py:658: UserWarning: 
Email address is not specified.

To make use of NCBI's E-utilities, NCBI requires you to specify your
email address with each request.  As an example, if your email address
is A.N.Other@example.com, you can specify it as follows:
   from Bio import Entrez
   Entrez.email = 'A.N.Other@example.com'
In case of excessive usage of the E-utilities, NCBI will attempt to contact
a user at the email address provided before blocking access to the
E-utilities.
  warnings.warn(


Sequence for P16888.2 changes from  to MPAKAQPAGKKGSKKAKAPRPSGGKKRRRRRKESYGIYIYKVLKQVHPDTGISSRAMSIMNSFVNDVFERIAAEASRLAHYNKKSTITSREVQTVVRLLLPGELAKHAVSEGTKAVTKYTTSK
Sequences updated: 1


In [23]:
cs.data[cs.data["variant"] == "CS_H2B"]

,accession,type,variant_group,variant,doublet,gi,ncbi_gene_id,hgnc_gene_name,taxonomy_id,organism,phylum,class,taxonomy_group,info,references,sequence
P16888.2,P16888.2,H2B,late_H2B,late_H2B.1,,,,,7668,Strongylocentrotus purpuratus,Echinodermata,Echinoidea,,,3697096,MPAKAQPAGKKGSKKAKAPRPSGGKKRRRRRKESYGIYIYKVLKQV...


In [24]:
cs.save()

Added sequence with accession P16888.2
cp histones.csv backups/histones.csv-Mar2324124212
Previous data backuped to backups/histones.csv-Mar2324124212
Results saved to histones.csv
